In [1]:
from sympy import symbols, Eq, solve, Matrix
import sympy as sp
import math

## Gregory Maju

In [2]:
data = [
    [1, 3.5, 6, 8.5],  # X
    [2.7, 3.32, 12.18, 20.1]  # Y
]
pembulatan = 5

x = 5
h = data[0][1] - data[0][0]
s = (x - data[0][0])/h
print("======================== COMPONENTS ========================")
print(f"x: {x}")
print(f"h: {h}")
print(f"s: {s:.5f}\n")

n = len(data[1])

# res = [
#     [2.7],
#     [3.32],
#     [12.18],
#     [20.1]
# ]
res = [[y] for y in data[1]]

# ===== Calculation Tables =====
for i in range(n-1):
    for j in range(n-i-1):
        curr = res[j][i]
        curr_forward = res[j+1][i]
        temp = curr_forward - curr
        res[j].append(temp)

print("======================== TABLE ========================")
for val in res:
    for v in val:
        print(f"{round(v, pembulatan):<12}", end="")
    print("")
print("")

# ===== Derivative Calculation =====

# Coefficient for derivative
derivative = res[0][1] / h  # Start with Δy₀ / h (first-order term)

# for i in range(2, len(res[0])):  # Start from the second-order term
#     term = res[0][i]  # Δ^i y₀
#     factorial = math.factorial(i)  # Factorial of the current order
#     # Calculate the coefficient (multiplier for the current term)
#     if i == 2:
#         coefficient = (2 * s - 1)
#     elif i == 3:
#         coefficient = (3 * s**2 - 6 * s + 2)
#     elif i == 4:
#         coefficient = (4 * s**3 - 18 * s**2 + 22 * s - 6)
#     else:
#         raise ValueError("Higher-order terms need to be explicitly defined.")
#     # Add the term to the derivative
#     derivative += (term * coefficient) / (factorial * h)

for i in range(2, len(res[0])):  # Start from the second-order term
    term = res[0][i]  # Δ^i y₀
    factorial = math.factorial(i)  # Factorial of the current order
    # Calculate the coefficient (multiplier for the current term)
    s_ = symbols('s')
    coefficient = s_
    for i in range(1, i):
        coefficient *= (s_ - i)
    coefficient = sp.diff(coefficient, s_)
    coefficient = coefficient.subs({s_: s}).evalf()
    derivative += (term * coefficient) / (factorial * h)


print("======================== RESULT ========================")
print(f"y'({x}) = {round(derivative, pembulatan)}")

======================== COMPONENTS ========================
x: 5
h: 2.5
s: 1.60000

======================== TABLE ========================
2.7         0.62        8.24        -9.18       
3.32        8.86        -0.94       
12.18       7.92        
20.1        

======================== RESULT ========================
y'(5) = 3.82464


## Gregory Mundur

## Lagrange